In [1]:
%load_ext autoreload
%autoreload 2


In [26]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from ml_logic.preprocessor import data_balancing, create_model, load_and_preprocess_image, image_augmentation, disease_categorization, num_cols, create_model_stage2


In [3]:
# Loading Datasets

train = pd.read_csv('../data/RFMiD_Training_Labels.csv').set_index('ID')
test = pd.read_csv('../data/RFMiD_Testing_Labels.csv').set_index('ID')
eval = pd.read_csv('../data/RFMiD_Validation_Labels.csv').set_index('ID')


In [4]:
# Splitting Data

X_train = disease_categorization(table_link='../data/')
y_train = X_train
X_eval  = eval.drop(columns='Disease_Risk')
y_eval = eval['Disease_Risk']


In [5]:
print(X_train.sample(5))
print(X_train.shape)
print(y_train.sample(5))
print(y_train.shape)


      DR  MH  DN  ODC  Others
ID                           
309    0   0   0    0       1
1541   1   0   0    0       0
1662   1   0   0    0       0
1671   0   0   0    0       1
1148   0   0   0    0       1
(1061, 5)
      DR  MH  DN  ODC  Others
ID                           
1370   0   0   0    1       0
226    0   0   1    0       0
1415   1   0   0    0       0
847    0   0   0    0       1
242    1   0   0    0       0
(1061, 5)


In [6]:
# Loading Images

image_folder = '../data/training_images'
eval_image_folder = '../data/eval_images'
images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_train.index])
eval_images = np.array([load_and_preprocess_image(row_id, eval_image_folder) for row_id in X_eval.index])


In [7]:
# plt.imshow(images[0])


In [8]:
# plt.imshow(eval_images[0])


In [9]:
print(images.shape)
print(eval_images.shape)


(1061, 224, 224, 3)
(640, 224, 224, 3)


In [10]:
# Augment Images

# augmented_images = image_augmentation(images)
# augmented_eval_images = image_augmentation(eval_images)
# print(augmented_images.shape)
# print(augmented_eval_images.shape)

# plt.imshow(augmented_images[0])
# plt.imshow(augmented_eval_images[0])


when comiling, it's categorical cross entropy instead of binary cross entropy

In [11]:
model = create_model_stage2((224,224,3), last_layer_neurons=num_cols(X_train))


2023-12-04 21:34:39.960758: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-04 21:34:39.961220: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-04 21:34:39.961437: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-R72GEULP): /proc/driver/nvidia/version does not exist
2023-12-04 21:34:39.969431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 774400)            0         
                                                                 
 dense (Dense)               (None, 12)                9292812   
                                                             

In [14]:
early_stopper = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.05,
    patience=10,
)


In [15]:
model.fit(
    augmented_images, y_train,
    validation_data=(augmented_eval_images, y_eval),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopper]
)


In [16]:
model.evaluate(augmented_eval_images, y_eval)
